In [1]:
import os
import sys
from pathlib import Path
import re
import multiprocessing
from typing import Callable, Optional
import pickle

from glob import glob
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, BertForPreTraining
import torch
import torch.nn as nn

import faiss

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
INPUT_DIR = 'chgk_baza'

def is_acceptable_quest(quest: dict) -> bool:
    if len(re.findall("\(pic: \d*.\w{1,3}\)", quest['Question'])) > 0:
        return False
    if len(re.findall("<раздатка>", quest['Question'])) > 0:
        return False
    return True

def pprint(iterable):
    for item in iterable:
        print(item)

def print_dict(data: dict):
    if data is not None:
        for key, value in data.items():
            print(f'{key}: {value}')

def parse_text(text):
    delim_regex = r'(Вопрос:\n|Ответ:\n|Комментарий:\n|Источник:\n|Автор:\n)'
    sections = re.split(delim_regex, "Вопрос:\n" + text)[1:]
    
    keys_replace_dict = {
        "Вопрос": "Question",
        "Ответ": "Answer",
        "Комментарий": "Comment",
        "Источник": "Source",
        "Автор": "Author",
    }

    keys = [keys_replace_dict[s.strip().strip(":")] for s in sections[0::2]]
    values = [s.strip().replace('\n', ' ') for s in sections[1::2]]
    return dict(zip(keys, values))

def preprocess_quest_dict(quest_dict: dict) -> Optional[dict]:
    result = dict()
    stop_set = {"---", "???"}
    if quest_dict.get('Question') is None or quest_dict.get('Answer') is None:
        return None
    if quest_dict['Question'] in stop_set or quest_dict['Answer'] in stop_set:
        return None
    if not is_acceptable_quest(quest_dict):
        return None
    
    replace_map = {"; .": ".", 
                   "  ": " ", 
                   "  ": " "}
    replace_map = dict((re.escape(k), v) for k, v in replace_map.items()) 
    pattern = re.compile("|".join(replace_map.keys()))
    result['Question'] = pattern.sub(lambda m: replace_map[re.escape(m.group(0))], quest_dict['Question'])

    replace_map = {"синонимичные ответы": "", 
                    "по смыслу": "", 
                    "и т.п.": "", 
                    "По смыслу.": "", 
                    "Зачет: ": "", 
                    "- и т.п. по смыслу": "", 
                    "Синонимичные ответы": "", 
                    "В любом написании." : "",
                    "В любом порядке.": "",
                    "В любом порядке": "",
                    "Незачет:": "",
                    "В любой орфографии.": "",
                    "Рейтинг:": "",
                    "В любом числе.": "",
                    "По упоминанию": "",
                    "По фамилии.": "",
                    "(засчитывается абсолютно точный ответ)": "",
                    ". .": ".", 
                    ".  .": ".", 
                    "; .": ".",
                    "   ": " ", 
                    "[": "",
                    "]": "",
                    '"': "",
                    ";": ",",
                    ", .": "."}
    
    replace_map = dict((re.escape(k), v) for k, v in replace_map.items()) 
    pattern = re.compile("|".join(replace_map.keys()))
    result['Answer'] = pattern.sub(lambda m: replace_map[re.escape(m.group(0))], quest_dict['Answer'])

    quest_dict['Answer'] = quest_dict['Answer'].split(' Тур: ')[0]

    if quest_dict.get('Comment') is not None:
        result['Comment'] = quest_dict['Comment']
    return result


In [3]:
files = sorted(list(Path(INPUT_DIR).rglob('*.txt')))

In [4]:
file_split_pattern = re.compile("Вопрос \d*:\n")

file = files[2]

with open(file, encoding='koi8-r') as infile:
    texts = re.split(file_split_pattern, infile.read())[1:]
    print(f"File: {file.name}, Len={len(texts)}\n")
    for text in texts:
        quest_dict = preprocess_quest_dict(parse_text(text))
        print_dict(quest_dict)
        print()


File: 120br3.txt, Len=120

Question: История из жизни.  В коридоре поликлиники сидят ожидающие своей очереди пациенты. Один из них встает, чтобы достать из заднего кармана зазвонивший телефон. Следом начинают подниматься остальные. Мелодию чего этот мужчина поставил на звонок?
Answer: Мелодию национального гимна.  Мелодию гимна страны, .

Question: Средневековые монашеские ордены зачастую практиковали обеты аскетизма и бескорыстия. Так, бенедиктинцам даже в обыденной речи запрещалось употреблять два слова. Кстати, по мнению автора "Слова о полку Игореве", употребление первого из этих слов спровоцировало братоубийственные распри, крамолы и войны. Назовите оба слова.
Answer: Мое, твое.  Мой, твой.
Comment: В "Слове о полку Игореве": "... стал спорить брат с братом: то мое, и это мое! И начали князи важным ничтожное звать и крамолу ковать друг на друга...".

Question: "Я бы не справился с чувством одиночества, - признаётся Мэтт, - я бы умер там в течение двадцати четырех часов". Где - там

In [5]:
texts = []
for file in tqdm(files):
    with open(file, encoding='koi8-r') as infile:
        texts = texts + re.split(file_split_pattern, infile.read())[1:]

quest_list = []
for text in tqdm(texts):
    quest = preprocess_quest_dict(parse_text(text))
    if quest is not None:
        quest_list.append(quest)


  0%|          | 0/4363 [00:00<?, ?it/s]

  0%|          | 0/336036 [00:00<?, ?it/s]

In [6]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def embed_bert_cls(text, model, tokenizer, max_len):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(DEVICE)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

def embed_bert_mean(text, model, tokenizer, max_len):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(DEVICE)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

def calc_texts_dist_cls(text1, text2, model, tokenizer, max_len) -> float:
    emb1 = embed_bert_cls(text1, model, tokenizer, max_len)
    emb2 = embed_bert_cls(text2, model, tokenizer, max_len)
    return ((emb1 - emb2)**2).sum()


def calc_embs(train_loader: DataLoader, model: nn.Module, tokenizer: AutoTokenizer, emb_function: Callable, max_len: int):
    embs = []
    for batch in tqdm(train_loader):
        # print(batch[1][0])
        question_batch, answer_batch = batch
        q_embs = emb_function(question_batch, model, tokenizer, max_len=max_len)
        a_embs = emb_function(answer_batch, model, tokenizer, max_len=max_len)
        embs.append(q_embs*0.5 + a_embs*0.5)
    return np.concatenate(embs)

In [7]:
# MODEL = "cointegrated/rubert-tiny2"
MODEL = "sberbank-ai/sbert_large_mt_nlu_ru"
EMB_FUNCTION = embed_bert_mean
MAX_LEN = 512

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModel.from_pretrained(MODEL).to(DEVICE)


In [8]:
# =========================================================================================
# Dataset
# =========================================================================================
class QuestNN_Dataset(Dataset):
    def __init__(self, quest_list: list):
        self.data = [self.quest_to_str(x) for x in quest_list]

    @classmethod
    def quest_to_str(cls, quest_dict: dict) -> str:
        # result = quest_dict['Question'] + "Ответ: " + quest_dict.get('Answer')
        # if quest_dict.get('Ответ') is not None:
        #     result += "Ответ: " + quest_dict.get('Ответ')
        # if quest_dict.get('Комментарий') is not None:
        #     result += quest_dict.get('Комментарий')
        return (quest_dict['Question'], 
                "Ответ: " + quest_dict.get('Answer'))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        return self.data[item]
    
os.environ["TOKENIZERS_PARALLELISM"] = "false"

train_loader = DataLoader(
    QuestNN_Dataset(quest_list), 
    batch_size = 256, 
    shuffle = False, 
    num_workers = 3, 
    pin_memory = True, 
    drop_last = False
)


In [9]:
embs = calc_embs(train_loader, model, tokenizer, EMB_FUNCTION, max_len=MAX_LEN)
# embs = calc_embs(train_loader, model, tokenizer, embed_bert_mean, max_len=MAX_LEN)

np.save(os.path.join(INPUT_DIR, 'embs_ruberttiny2.npy'), embs)

  0%|          | 0/1250 [00:00<?, ?it/s]

In [9]:
embs = np.load(os.path.join(INPUT_DIR, 'embs.npy'))
print(embs.shape)
# (319759, 312)

(319759, 1024)


In [10]:
#Sentences we want sentence embeddings for
sentences = np.array([QuestNN_Dataset.quest_to_str(x) for x in quest_list[10:12]])
print(sentences)

#Perform pooling. In this case, mean pooling
sentence_embeddings = EMB_FUNCTION(list(sentences[:, 0]), model, tokenizer, max_len=MAX_LEN)*0.5 + EMB_FUNCTION(list(sentences[:, 1]), model, tokenizer, max_len=MAX_LEN)*0.5
sentence_embeddings = embed_bert_mean(list(sentences[:, 0]), model, tokenizer, max_len=MAX_LEN)*0.5 + embed_bert_mean(list(sentences[:, 1]), model, tokenizer, max_len=MAX_LEN)*0.5

assert embs.shape[1] == sentence_embeddings.shape[1]
print((sentence_embeddings[0] * sentence_embeddings[1]).sum())
print((embs[10] * embs[11]).sum())

[['В одной из серий "Доктора Кто", действие которой разворачивается в 1926 году, ОНА на вопрос о своем выборе отвечает так: бельгийцы пекут вкусные булочки. Другая "ОНА" появилась в 1988 году. Назовите ЕЕ.'
  'Ответ: Агата Кристи.']
 ['Согласно юмористическому сообщению, неизвестная русская женщина парализовала проходившие в графстве Эссекс соревнования... По какому спорту?'
  'Ответ: По конному спорту.']]
0.40359873
0.40359876


In [11]:
res = faiss.StandardGpuResources()
nlist = 5000
index = faiss.index_cpu_to_gpu(res, 0, faiss.IndexFlatL2(embs.shape[1]))
quantizer = faiss.IndexFlatL2(embs.shape[1])
# index = faiss.IndexIVFFlat(quantizer, embs.shape[1], nlist)
# gpu_index = faiss.index_cpu_to_gpu(res, 0, index)
if not index.is_trained:
    index.train(embs)
index.add(embs)                  
# faiss.write_index(index, os.path.join(INPUT_DIR, 'faiss.index'))

In [12]:
print(faiss.MatrixStats(embs).comments)
# index = faiss.read_index(os.path.join(INPUT_DIR, 'faiss.index'))
k = 15
distances, indices = index.search(embs, k) # sanity check
print(indices[:, 1:])
print(distances[:, 1:])

analyzing 319759 vectors of size 1024
no NaN or Infs in data
318598 vectors are distinct (99.64%)
vector 210192 has 3 copies
range of L2 norms=[0.683066, 0.984089] (0 null vectors)
matrix contains 0.00 % 0 entries
no constant dimensions
no dimension has a too large mean
stddevs per dimension are in [0.00707198 0.0475416]

[[140816 181004 314568 ... 200632 136512  52933]
 [299021  74863 191326 ... 257580 228281 104425]
 [220792 117175 117155 ... 237793 252345  73565]
 ...
 [122475   4844 183200 ... 256889 207840 203193]
 [201792 223845 151514 ... 294441 200230 260271]
 [183390  91982 255115 ... 146911 230299 185381]]
[[0.21680605 0.23153412 0.24172926 ... 0.2721287  0.27239072 0.2754647 ]
 [0.14605105 0.15370452 0.15524256 ... 0.1721797  0.17217982 0.17256498]
 [0.23815942 0.24166662 0.24699628 ... 0.2724374  0.27262688 0.2733537 ]
 ...
 [0.195647   0.20073068 0.20313627 ... 0.2142762  0.21428531 0.21459097]
 [0.19521976 0.19617343 0.20455182 ... 0.2204575  0.2213288  0.22217023]
 [0.17

In [15]:
quest_dict = dict()
i = 0

for quest in quest_list:
    quest_dict[i] = (quest, indices[:, 1:][i], distances[:, 1:][i])
    i += 1
    
print(len(quest_dict))

threshold = 0.12
for i in tqdm(range(len(quest_dict))):
    if quest_dict.get(i) is not None:
        quest, q_indices, q_distances = quest_dict[i]
        for ind, dist in zip(q_indices, q_distances):
            if quest_dict.get(ind) is not None:
                if dist < threshold:
                    # print(quest_dict[i], "\n", quest_dict[ind], "\n")
                    quest_dict.pop(ind, None)

print(len(quest_dict))

319759


  0%|          | 0/319759 [00:00<?, ?it/s]

291735


In [16]:
threshold = 0.1203
for i in tqdm(range(len(quest_dict))):
    if quest_dict.get(i) is not None:
        quest, q_indices, q_distances = quest_dict[i]
        for ind, dist in zip(q_indices, q_distances):
            if quest_dict.get(ind) is not None:
                if dist < threshold:
                    print(quest_dict[i][0], "\n", quest_dict[ind][0], "\n")
                    quest_dict.pop(ind, None)

  0%|          | 0/291735 [00:00<?, ?it/s]

{'Question': 'ЕМУ приписывают фразу: "Главное, чтобы тебя окружали хорошие люди". По легенде, в начале 1943 года Гитлер лично положил на ЕГО гроб жезл с бриллиантами. Назовите ЕГО.', 'Answer': 'Фридрих Паулюс.'} 
 {'Question': 'Во время воздушного налета ОН потерял глаз, правую руку и два пальца левой руки. В 1943 году ОН писал жене: "Настанет время, когда я спасу Германию". Назовите ЕГО.', 'Answer': 'Клаус Шенк фон Штауффенберг.'} 

{'Question': 'Какое существо Плиний Старший определил так: "змея с белым пятном на голове, похожим на корону или диадему"?', 'Answer': 'василиск.'} 
 {'Question': 'Согласно Лукану, из крови Медузы Горгоны появился царь Ливийских змей. Под каким названием он фигурирует у Плиния Старшего?', 'Answer': 'Василиск.', 'Comment': '"Царек" по-гречески. Подобно Медузе Горгоне, василиск обладал смертоносным взглядом.'} 

{'Question': 'Оно долгое время вызывало многочисленные нарекания. Например, в 1913 году кайзер Вильгельм II запретил своим офицерам позорить мундир,

In [17]:
quest_clean_list = []
for _, quest in quest_dict.items():
    quest_clean_list.append(quest[0])

print(len(quest_clean_list))
with open(os.path.join(INPUT_DIR, 'quest_clean_list.pkl'), 'wb') as f:
    pickle.dump(quest_clean_list, f, protocol=pickle.HIGHEST_PROTOCOL)

291616


In [18]:
with open(os.path.join(INPUT_DIR, 'quest_clean_list.pkl'), 'rb') as f:
    quest_clean_list = pickle.load(f)